### Создание кастомной модели FastText для поддержания диалога чат-ботом

##### 1. Импорт библиотек

In [1]:
import pandas as pd
import string
import emoji
import re
from tqdm import tqdm
from stop_words import get_stop_words
import razdel
import nltk
from pymorphy2 import MorphAnalyzer
from gensim.models import FastText
import numpy as np
import random
import dill
tqdm.pandas()

C:\Users\lazar\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


##### 2. Подготовка к препроцессингу датасета

In [2]:
# Размерность вектора эмбеддингов для слов
embedding_dim=128

In [3]:
# Импорт знаков пунктуации для препроцессинга
punctuations = string.punctuation
punctuations

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [4]:
# Импорт Стоп-слов
stop_words = get_stop_words('ru')
stops = nltk.corpus.stopwords.words('russian')
stop_words.extend(stops)
stop_words = list(set(stop_words))
print(f'Всего найдено {len(stop_words)} стоп-слов')

Всего найдено 422 стоп-слов


In [5]:
morph_analyzer = MorphAnalyzer()

##### Чтение датасетов:

In [ ]:
# 1й датасет
df = pd.read_csv(r'D:\Chat-bot\talker\talker.csv', sep='\t', encoding='utf-8')
df.drop(columns=['Unnamed: 0', 'short_phrase'], inplace=True)
df.rename(columns={'expanded_phrase':'answer', 'context':'question'}, inplace=True)
df['answer'] = df['answer'].apply(lambda x: f"{x[0].upper()}{x[1:]}")

In [ ]:
# 3й датасет
df_3 = pd.read_csv(r'D:\Chat-bot\talker\talker_3.csv', sep='\t', encoding='utf-8')
df_3.drop(columns=['Unnamed: 0'], inplace=True)
df_3['answer'] = df_3['answer'].apply(lambda x: f"{str(x)[0].upper()}{str(x)[1:]}")

In [ ]:
#df = pd.concat([df, df_2, df_3], axis=0, ignore_index=True)
df = pd.concat([df, df_3], axis=0, ignore_index=True)

In [ ]:
df = df[df['question'].notnull()]
df = df[df['answer'].notnull()]
df

##### 3. Препроцессинг: удаление знаков пунктуаций, эмоджи, чисел и т.д.

In [6]:
def preprocess_text(text:str, punctuations=punctuations, stop_words=stop_words, morph=morph_analyzer):
    
    text = str(text)
    text = text.lower()                 # Изменяем регистр на нижний
    
    # Замена пунктуации пробелами:
    for el in string.punctuation:
        text = text.replace(el, ' ')
    
#    # Замена специальных символов на пробелы
#    text = re.sub(r'[^a-Za-Z0-9]', ' ', text)
#    print(text)

    # Удаляем эмоджи из текста
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'a', text)
    
    # Сличение частицы "не" с глаголом
    text = re.sub(r'не ', r'не', text)

    # Замена чисел на пробелы
    text = re.sub(r'\d+', ' ', text)
   
    # Удаляем слова длиной в 1 символ
    # Удаляем слова из словаря стоп-слов
    # Получаем нормальную форму слова
    text = ' '.join([morph.parse(w)[0].normal_form for w in text.split() if len(w)>1 and w not in stop_words])
  
    return text

In [ ]:
# Препроцессинг
df['question_preprocessed'] = df['question'].progress_apply(lambda x: preprocess_text(x))

In [ ]:
# Токенезация
df['question_tokenized'] = df['question_preprocessed'].progress_apply(lambda x: [_.text for _ in  list(razdel.tokenize(x))])

In [ ]:
# Максимальное количество значимых токенов в предложении
df['количество токенов'] = df['question_tokenized'].apply(lambda x: len(x))
df['количество токенов'].max()

In [ ]:
# Удаление предложений с количеством значимых токенов = 0
df = df[df['количество токенов'] > 0]
df.reset_index(drop=True, inplace=True)
df.shape

In [ ]:
# Добавляем слова-ассоциации:
associations = pd.read_csv(r'D:\Chat-bot\associations\associations.csv', encoding='utf-8', sep='\t')
associations.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
associations['question_preprocessed'] = associations['0'].progress_apply(lambda x: preprocess_text(x))
associations['question_tokenized'] = associations['question_preprocessed'].progress_apply(lambda x: [_.text for _ in list(razdel.tokenize(x))])
associations['количество токенов'] = associations['question_tokenized'].apply(lambda x: len(x))
associations

In [ ]:
sentences = [el for el in df[df['количество токенов'] > 1]['question_tokenized'].values.tolist()]
sentences.extend([el for el in associations[associations['количество токенов'] > 1]['question_tokenized'].values.tolist()])

##### 4. Сохранение подготовленного датасета на диск

In [ ]:
with open(r'D:\Chat-bot\talker\df_preprocessed.dill', 'wb') as f:
    dill.dump(df, f)

In [7]:
with open(r'D:\Chat-bot\talker\df_preprocessed.dill', 'rb') as f:
    df = dill.load(f)

##### 5. Обучение модели FastText. Сохранение обученной модели на диск

In [ ]:
model = FastText(sentences=sentences, vector_size=embedding_dim, window=2, min_count=1)  # instantiate
#model.build_vocab(corpus_iterable=df['question_tokenized'].values.tolist())
model.train(corpus_iterable=df[df['количество токенов'] > 1]['question_tokenized'].values.tolist(), total_examples=df.shape[0], epochs=10)  # train

In [ ]:
model.save(r'D:\Chat-bot\talker\fastetx_talker_model_2.fst')

##### 6. Получение эмбеддинга запроса пользователя. Получение наиболее релевантного ответа

In [9]:
model = FastText.load(r'D:\Chat-bot\talker\fastetx_talker_model_2.fst')

In [10]:
def words_to_embeddings(mas:list):
    vector = np.zeros(embedding_dim)
    i = 0
    for word in mas:
        i += 1
        vector += model.wv[word]
    return vector/i
        
df['sentence_embeddings'] = df['question_tokenized'].progress_apply(lambda x: words_to_embeddings(x))

100%|██████████| 136054/136054 [00:03<00:00, 40427.45it/s]


In [11]:
def compute_distance_between_vectors(i, j):
    dot_product = np.dot(i, j)
    magnitude1 = np.linalg.norm(i)
    magnitude2 = np.linalg.norm(j)
    angle = np.arccos(dot_product / (magnitude1 * magnitude2))
    return angle

In [16]:
text = 'Привет! Как дела?'
text=preprocess_text(text)
text = [_.text for _ in list(razdel.tokenize(text))]
text = words_to_embeddings(text)

computing = df['sentence_embeddings'].progress_apply(lambda x: compute_distance_between_vectors(x, text))
index = random.choice(computing.sort_values(ascending=True).head(5).index)
df.loc[index, 'answer']

100%|██████████| 136054/136054 [00:01<00:00, 73268.54it/s]


'Привет! Все хорошо, спасибо. А у тебя?'